#### Dataset Link - https://www.kaggle.com/mlg-ulb/creditcardfraud

In [35]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/content/drive/MyDrive/creditcard.csv')
df.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,-1.593105,2.711941,-0.689256,4.626942,-0.924459,1.107641,1.991691,0.510632,-0.682920,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,-0.150189,0.915802,1.214756,-0.675143,1.164931,-0.711757,-0.025693,-1.221179,-1.545556,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,0.411614,0.063119,-0.183699,-0.510602,1.329284,0.140716,0.313502,0.395652,-0.577252,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,-1.933849,-0.962886,-1.042082,0.449624,1.962563,-0.608577,0.509928,1.113981,2.897849,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0
284806,172792.0,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,-0.915427,-1.040458,-0.031513,-0.188093,-0.084316,0.041333,-0.302620,-0.660377,0.167430,-0.256117,0.382948,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,217.00,0


In [4]:
df.shape

(284807, 31)

In [5]:
df.isna().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [6]:
df.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [7]:
X = df.drop('Class', axis=1)
Y = df.Class

### Cross Validation KFold & Hyperparameter Tunning

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import KFold, GridSearchCV

In [9]:
log_class = LogisticRegression()
grid= {'C':10.0**np.arange(-2,3), 'penalty':['11','l2']}
cv = KFold(n_splits= 5, shuffle= False, random_state=None)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, train_size=0.7)

In [12]:
cl = GridSearchCV(log_class,grid,cv=cv, n_jobs=-1,scoring='f1_macro')
cl.fit(X_train,Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'penalty': ['11', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=0)

In [15]:
Y_pred = cl.predict(X_test)
print("Accuracy Score", accuracy_score(Y_test, Y_pred))
print( confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

Accuracy Score 0.9989700736163291
[[85267    33]
 [   55    88]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85300
           1       0.73      0.62      0.67       143

    accuracy                           1.00     85443
   macro avg       0.86      0.81      0.83     85443
weighted avg       1.00      1.00      1.00     85443



In [20]:
class_weight = dict({0:1, 1:100})

In [21]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(class_weight = class_weight)
classifier.fit(X_train,Y_train)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight={0: 1, 1: 100}, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

#### Without class_weight 

In [19]:
Y_pred = classifier.predict(X_test)
print("Accuracy Score", accuracy_score(Y_test, Y_pred))
print( confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

Accuracy Score 0.9994616293903538
[[85291     9]
 [   37   106]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85300
           1       0.92      0.74      0.82       143

    accuracy                           1.00     85443
   macro avg       0.96      0.87      0.91     85443
weighted avg       1.00      1.00      1.00     85443



#### With Class Weight

In [22]:
Y_pred = classifier.predict(X_test)
print("Accuracy Score", accuracy_score(Y_test, Y_pred))
print( confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

Accuracy Score 0.9994499256814484
[[85295     5]
 [   42   101]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85300
           1       0.95      0.71      0.81       143

    accuracy                           1.00     85443
   macro avg       0.98      0.85      0.91     85443
weighted avg       1.00      1.00      1.00     85443



#### Undersampling

In [24]:
Y_train.value_counts()

0    199015
1       349
Name: Class, dtype: int64

In [28]:
from collections import Counter
from imblearn.under_sampling import NearMiss
ns = NearMiss(0.8)
X_train_ns, Y_train_ns = ns.fit_sample(X_train,Y_train)
print("Number of classes before fit{}".format(Counter(Y_train)))
print("Number of classes after fit{}".format(Counter(Y_train_ns)))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Number of classes before fitCounter({0: 199015, 1: 349})
Number of classes after fitCounter({0: 436, 1: 349})


In [29]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train_ns,Y_train_ns)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [30]:
Y_pred = classifier.predict(X_test)
print("Accuracy Score", accuracy_score(Y_test, Y_pred))
print( confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

Accuracy Score 0.773544936390342
[[65966 19334]
 [   15   128]]
              precision    recall  f1-score   support

           0       1.00      0.77      0.87     85300
           1       0.01      0.90      0.01       143

    accuracy                           0.77     85443
   macro avg       0.50      0.83      0.44     85443
weighted avg       1.00      0.77      0.87     85443



#### Oversampling

In [31]:
from imblearn.over_sampling import RandomOverSampler

In [32]:
os = RandomOverSampler(0.75)
X_train_ns, Y_train_ns = os.fit_sample(X_train,Y_train)
print("Number of classes before fit{}".format(Counter(Y_train)))
print("Number of classes after fit{}".format(Counter(Y_train_ns)))

Number of classes before fitCounter({0: 199015, 1: 349})
Number of classes after fitCounter({0: 199015, 1: 99507})


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [33]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train_ns,Y_train_ns)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [34]:
Y_pred = classifier.predict(X_test)
print("Accuracy Score", accuracy_score(Y_test, Y_pred))
print( confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

Accuracy Score 0.9994499256814484
[[85292     8]
 [   39   104]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85300
           1       0.93      0.73      0.82       143

    accuracy                           1.00     85443
   macro avg       0.96      0.86      0.91     85443
weighted avg       1.00      1.00      1.00     85443



#### SMOTETomek

In [36]:
from imblearn.combine import SMOTETomek

In [38]:
os = SMOTETomek()
X_train_ns, Y_train_ns = os.fit_sample(X_train,Y_train)
print("Number of classes before fit{}".format(Counter(Y_train)))
print("Number of classes after fit{}".format(Counter(Y_train_ns)))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Number of classes before fitCounter({0: 199015, 1: 349})
Number of classes after fitCounter({0: 198383, 1: 198383})


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [39]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train_ns,Y_train_ns)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [40]:
Y_pred = classifier.predict(X_test)
print("Accuracy Score", accuracy_score(Y_test, Y_pred))
print( confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

Accuracy Score 0.999403110845827
[[85281    19]
 [   32   111]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85300
           1       0.85      0.78      0.81       143

    accuracy                           1.00     85443
   macro avg       0.93      0.89      0.91     85443
weighted avg       1.00      1.00      1.00     85443

